<a href="https://colab.research.google.com/github/ajimper/ElAstronomoCriollo/blob/main/DualHR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install astroquery

In [ ]:
from astroquery.vizier import Vizier
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt

# Configurar consultas a Vizier
Vizier.ROW_LIMIT = -1  # Descargar todos los resultados
Vizier.columns = ['all']  # Seleccionar todas las columnas

class Star:
    def __init__(self, data):
        self.data = data
        self._spectral_teff = {
            'O': 35000, 'B': 20000, 'A': 8500,
            'F': 6500, 'G': 5500, 'K': 4400, 'M': 3200
        }

    @property
    def teff(self):
        """Calcula temperatura efectiva usando B-V o tipo espectral"""
        if 'B-V' in self.data.colnames and not np.isnan(self.data['B-V']):
            return 4600 * (1/(0.92 * self.data['B-V'] + 1.7) + 1/(0.92 * self.data['B-V'] + 0.62)) * u.K
        else:
            sp_type = self.data['SpType'][0] if 'SpType' in self.data.colnames else 'G'
            return self._spectral_teff.get(sp_type, 5500) * u.K

    @property
    def distance(self):
        """Calcula distancia con corrección de paralaje"""
        plx = self.data['Plx'] if 'Plx' in self.data.colnames else self.data['Parallax']
        if plx <= 0:
            return np.nan * u.pc
        return (1000/plx) * u.pc

    @property
    def absolute_magnitude(self):
        """Magnitud absoluta en banda V"""
        if 'Vmag' in self.data.colnames:
            return self.data['Vmag'] - 5 * np.log10(self.distance.value/10)
        return np.nan

    @property
    def luminosity(self):
        """Luminosidad bolométrica"""
        bc = -0.2 * self.data['B-V'] if 'B-V' in self.data.colnames else -0.5
        m_bol = self.absolute_magnitude + bc
        return 10**(-0.4*(m_bol - 4.74)) * u.Lsun

# Versión 1: Catálogo Hipparcos
def plot_hipparcos():
    catalog = Vizier.get_catalogs('I/239/hip_main')[0]
    named_stars = catalog[~catalog['ProperName'].mask]

    stars = [Star(row) for row in named_stars]
    teff = [s.teff.value for s in stars if not np.isnan(s.distance.value)]
    lum = [s.luminosity.value for s in stars if not np.isnan(s.distance.value)]
    names = [s.data['ProperName'] for s in stars if not np.isnan(s.distance.value)]

    plt.figure(figsize=(12,8))
    plt.scatter(teff, lum, c='royalblue', alpha=0.7)
    plt.xlim(30000, 2000)
    plt.yscale('log')
    plt.title('Diagrama HR - Hipparcos (Estrellas con nombre)')
    plt.xlabel('Temperatura Efectiva (K)')
    plt.ylabel('Luminosidad (L☉)')

    # Anotar algunos nombres
    for i, name in enumerate(names):
        if lum[i] > 1000 or teff[i] > 10000:
            plt.annotate(name, (teff[i], lum[i]), fontsize=8)
    plt.show()

# Versión 2: Bright Star Catalogue
def plot_bright_star():
    catalog = Vizier.get_catalogs('V/50')[0]
    named_stars = catalog[~catalog['Name'].mask]

    stars = [Star(row) for row in named_stars]
    teff = [s.teff.value for s in stars if not np.isnan(s.distance.value)]
    lum = [s.luminosity.value for s in stars if not np.isnan(s.distance.value)]
    names = [s.data['Name'] for s in stars if not np.isnan(s.distance.value)]

    plt.figure(figsize=(12,8))
    plt.scatter(teff, lum, c='crimson', alpha=0.7)
    plt.xlim(30000, 2000)
    plt.yscale('log')
    plt.title('Diagrama HR - Bright Star Catalogue')
    plt.xlabel('Temperatura Efectiva (K)')
    plt.ylabel('Luminosidad (L☉)')

    # Anotar nombres brillantes
    for i, name in enumerate(names):
        if lum[i] > 100 or teff[i] < 4000:
            plt.annotate(name, (teff[i], lum[i]), fontsize=8)
    plt.show()

# Ejecutar ambas versiones
plot_hipparcos()
plot_bright_star()